# Step 1: Setting Up the Environment and Connecting to WRDS


First, you need to establish a connection to WRDS through your JupyterHub instance. Let's import the required libraries and connect to WRDS.

In [1]:
import os.path

import wrds
import pandas as pd
import numpy as np


# Connect to WRDS
conn = wrds.Connection()


WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


# Step 2: Define the Time Window and Set Parameters

In the SAS code, dates were defined for the study period (begindate and enddate). Let's define them in Python.

In [4]:
# Define study period (as in the original SAS script)
begin_date = '2006-01-01'
end_date = '2019-12-31'

# Print the date range for verification
print(f"Study period: {begin_date} to {end_date}")

Study period: 2006-01-01 to 2019-12-31


# Step 3: Extract Data from Compustat

The SAS code extracts various variables from Compustat using proc sql. In Python, we will use SQL queries through wrds.Connection() to obtain this data.

In [5]:
# SQL query to extract data from Compustat's Quarterly Fundamentals (FUNDQ) database
compustat_query = f"""
    SELECT gvkey, fyearq, fqtr, conm, datadate, rdq, epsfxq, epspxq, prccq, ajexq, spiq, 
           cshoq, cshprq, cshfdq, saleq, atq, fyr, consol, indfmt, datafmt, popsrc, datafqtr
    FROM comp.fundq
    WHERE fyr > 0 
      AND (saleq > 0 OR atq > 0)
      AND consol = 'C' 
      AND popsrc = 'D' 
      AND indfmt = 'INDL' 
      AND datafmt = 'STD'
      AND datadate BETWEEN '{begin_date}' AND '{end_date}'
"""

# Extract data using the connection
compustat_data = conn.raw_sql(compustat_query)

# Show the first few rows of the extracted data for validation
compustat_data.head()


,gvkey,fyearq,fqtr,conm,datadate,rdq,epsfxq,epspxq,prccq,ajexq,...,cshprq,cshfdq,saleq,atq,fyr,consol,indfmt,datafmt,popsrc,datafqtr
0,001013,2006,1.0,ADC TELECOMMUNICATIONS INC,2006-01-31,2006-03-01,-0.01,-0.01,25.360,1.00,...,116.700,116.700,272.800,1514.700,10,C,INDL,STD,D,2006Q1
1,001082,2005,3.0,SERVIDYNE INC,2006-01-31,2006-03-15,-0.08,-0.08,4.111,1.05,...,3.531,3.531,4.321,51.833,4,C,INDL,STD,D,2005Q3
2,001173,2005,4.0,AEROSONIC CORP,2006-01-31,2006-03-28,0.29,0.29,7.000,1.00,...,3.925,3.930,7.754,20.212,1,C,INDL,STD,D,2005Q4
3,001183,2005,4.0,IDNA INC,2006-01-31,2006-05-15,-0.03,-0.03,0.370,1.00,...,8.839,8.839,5.091,28.847,1,C,INDL,STD,D,2005Q4
4,001240,2005,4.0,ALBERTSON'S INC,2006-01-31,2006-03-07,0.44,0.44,25.150,1.00,...,370.000,373.000,10227.000,17871.000,1,C,INDL,STD,D,2005Q4


# Step 4: Extract Data from IBES

The next step in the SAS code is to extract relevant data from IBES. 

In [7]:
# Inspect the available columns in the IBES dataset
ibes_metadata = conn.get_table('ibes', 'statsum_epsus')
ibes_metadata.columns

Index(['ticker', 'cusip', 'oftic', 'cname', 'statpers', 'measure', 'fiscalp',
       'fpi', 'estflag', 'curcode', 'numest', 'numup', 'numdown', 'medest',
       'meanest', 'stdev', 'highest', 'lowest', 'usfirm', 'fpedats', 'actual',
       'actdats_act', 'acttims_act', 'anndats_act', 'anntims_act', 'curr_act'],
      dtype='object')

In [8]:
# Updated SQL query to extract data from IBES
ibes_query_updated = f"""
    SELECT ticker, actual, fpedats, anndats_act, measure, fpi, meanest, medest, numest, usfirm
    FROM ibes.statsum_epsus
    WHERE measure = 'EPS' 
      AND fpi IN ('6', '7') 
      AND fpedats BETWEEN '{begin_date}' AND '{end_date}'
      AND anndats_act IS NOT NULL
"""

# Extract data using the connection
ibes_data_updated = conn.raw_sql(ibes_query_updated)

# Show the first few rows of the extracted data for validation
ibes_data_updated.head()

,ticker,actual,fpedats,anndats_act,measure,fpi,meanest,medest,numest,usfirm
0,0000,0.12,2014-03-31,2014-05-06,EPS,6,0.08,0.07,4.0,1
1,0000,0.27,2014-06-30,2014-08-06,EPS,6,0.13,0.13,5.0,1
2,0000,0.27,2014-06-30,2014-08-06,EPS,6,0.13,0.13,5.0,1
3,0000,0.27,2014-06-30,2014-08-06,EPS,6,0.13,0.13,5.0,1
4,0000,0.27,2014-06-30,2014-08-06,EPS,7,0.12,0.11,5.0,1


# Step 5: Link IBES Data with Compustat Data

Since we do not have the ibes.iclink table, we'll use a multi-step linking approach:

Step 5.1: Extract IBES Ticker and Other Data from the relevant dataset.
Step 5.2: Use CRSP to find the mapping between Ticker and Permno.
Step 5.3: Use CRSP/Compustat Merged (CCM) to link Permno and GVKEY.
Here is how you can proceed step by step:

## Step 5.1: Extract Data from IBES with the Ticker

We have already extracted data from the IBES dataset with tickers (ibes_data_updated). Now we will proceed with linking it through CRSP and CCM.

In [10]:
# ibes_data_updated already has ticker, actual, etc.
ibes_data_updated.head()

,ticker,actual,fpedats,anndats_act,measure,fpi,meanest,medest,numest,usfirm
0,0000,0.12,2014-03-31,2014-05-06,EPS,6,0.08,0.07,4.0,1
1,0000,0.27,2014-06-30,2014-08-06,EPS,6,0.13,0.13,5.0,1
2,0000,0.27,2014-06-30,2014-08-06,EPS,6,0.13,0.13,5.0,1
3,0000,0.27,2014-06-30,2014-08-06,EPS,6,0.13,0.13,5.0,1
4,0000,0.27,2014-06-30,2014-08-06,EPS,7,0.12,0.11,5.0,1


## Step 5.2: Link to CRSP Using Ticker

We will use a table from the CRSP database to link the ticker to permno. In CRSP, the stocknames table contains a mapping between tickers, CUSIPs, and PERMNOs.

In [11]:
# SQL query to extract CRSP ticker and permno
crsp_query = f"""
    SELECT permno, ticker, namedt, nameenddt
    FROM crsp.stocknames
    WHERE namedt <= '{end_date}' AND nameenddt >= '{begin_date}'
"""

# Extract data using the connection
crsp_data = conn.raw_sql(crsp_query)

# Show the first few rows for verification
crsp_data.head()


,permno,ticker,namedt,nameenddt
0,10001,EWST,1993-11-22,2008-02-04
1,10001,EWST,2008-02-05,2009-08-03
2,10001,EGAS,2009-08-04,2009-12-17
3,10001,EGAS,2009-12-18,2010-07-08
4,10001,EGAS,2010-07-09,2017-08-03


Now we will merge the CRSP data with the IBES data based on the ticker. We will use the namedt and nameenddt fields to ensure that the data matches within the appropriate date ranges.

In [12]:
# Merge IBES and CRSP data on ticker and date range
ibes_crsp_linked = pd.merge(
    ibes_data_updated,
    crsp_data,
    on='ticker',
    how='inner'
)

# Filter by date to ensure namedt and nameenddt match the forecast period
ibes_crsp_linked = ibes_crsp_linked[
    (ibes_crsp_linked['fpedats'] >= ibes_crsp_linked['namedt']) &
    (ibes_crsp_linked['fpedats'] <= ibes_crsp_linked['nameenddt'])
]

# Show the first few rows of the merged data for validation
ibes_crsp_linked.head()


,ticker,actual,fpedats,anndats_act,measure,fpi,meanest,medest,numest,usfirm,permno,namedt,nameenddt
2,AA,2.1,2006-03-31,2006-04-10,EPS,6,1.38,1.41,15.0,1,24643,1999-01-04,2016-10-05
6,AA,2.1,2006-03-31,2006-04-10,EPS,6,1.42,1.38,14.0,1,24643,1999-01-04,2016-10-05
10,AA,2.1,2006-03-31,2006-04-10,EPS,6,1.53,1.56,13.0,1,24643,1999-01-04,2016-10-05
14,AA,2.1,2006-03-31,2006-04-10,EPS,7,1.22,1.19,10.0,1,24643,1999-01-04,2016-10-05
18,AA,2.1,2006-03-31,2006-04-10,EPS,7,1.25,1.20,10.0,1,24643,1999-01-04,2016-10-05


## Step 5.3: Link CRSP to Compustat (Using PERMNO and GVKEY)

Next, we will link the permno from CRSP to gvkey using the CRSP/Compustat Merged (CCM) database.

In [13]:
# SQL query to extract CRSP-Compustat linking information
ccm_query = f"""
    SELECT gvkey, lpermno AS permno, linkdt, linkenddt
    FROM crsp.ccmxpf_linktable
    WHERE linkdt <= '{end_date}' AND (linkenddt >= '{begin_date}' OR linkenddt IS NULL)
"""

# Extract data using the connection
ccm_data = conn.raw_sql(ccm_query)

# Show the first few rows of the linking data for validation
ccm_data.head()


,gvkey,permno,linkdt,linkenddt
0,001003,NaN,1989-08-17,None
1,001004,54594.0,1972-04-24,None
2,001009,NaN,1996-03-14,2007-02-28
3,001010,NaN,1984-06-29,None
4,001013,50906.0,1979-03-16,2010-12-31


Now merge the IBES-CRSP linked data with the CCM linking information to get gvkey.

In [14]:
# Merge IBES-CRSP linked data with CCM linking table on permno
final_linked_data = pd.merge(
    ibes_crsp_linked,
    ccm_data,
    on='permno',
    how='inner'
)

# Filter by date to ensure linkdt and linkenddt match the forecast period
final_linked_data = final_linked_data[
    (final_linked_data['fpedats'] >= final_linked_data['linkdt']) &
    ((final_linked_data['fpedats'] <= final_linked_data['linkenddt']) | final_linked_data['linkenddt'].isna())
]

# Show the first few rows of the final linked data for validation
final_linked_data.head()


,ticker,actual,fpedats,anndats_act,measure,fpi,meanest,medest,numest,usfirm,permno,namedt,nameenddt,gvkey,linkdt,linkenddt
0,AA,2.1,2006-03-31,2006-04-10,EPS,6,1.38,1.41,15.0,1,24643,1999-01-04,2016-10-05,001356,1962-01-31,2016-10-31
2,AA,2.1,2006-03-31,2006-04-10,EPS,6,1.42,1.38,14.0,1,24643,1999-01-04,2016-10-05,001356,1962-01-31,2016-10-31
4,AA,2.1,2006-03-31,2006-04-10,EPS,6,1.53,1.56,13.0,1,24643,1999-01-04,2016-10-05,001356,1962-01-31,2016-10-31
6,AA,2.1,2006-03-31,2006-04-10,EPS,7,1.22,1.19,10.0,1,24643,1999-01-04,2016-10-05,001356,1962-01-31,2016-10-31
8,AA,2.1,2006-03-31,2006-04-10,EPS,7,1.25,1.20,10.0,1,24643,1999-01-04,2016-10-05,001356,1962-01-31,2016-10-31


# Step 6: Merge the Compustat Data with the Final Linked Data

The final linked dataset (final_linked_data) contains data from IBES, CRSP, and a linking gvkey which we can merge with Compustat data. Here, we are going to merge the Compustat quarterly data (compustat_data) with the linked IBES-CRSP-Compustat data (final_linked_data) based on gvkey.

In [15]:
# Compustat data was extracted in Step 3
compustat_data.head()

# Merge Compustat data with the final linked dataset on 'gvkey'
# We use 'inner' join to ensure we only keep matched data
final_dataset = pd.merge(
    final_linked_data,
    compustat_data,
    on='gvkey',
    how='inner'
)

# After merging, let's check for the first few rows to validate the data
final_dataset.head()


,ticker,actual,fpedats,anndats_act,measure,fpi,meanest,medest,numest,usfirm,...,cshprq,cshfdq,saleq,atq,fyr,consol,indfmt,datafmt,popsrc,datafqtr
0,AA,2.1,2006-03-31,2006-04-10,EPS,6,1.38,1.41,15.0,1,...,870.561,875.972,7111.0,34929.0,12,C,INDL,STD,D,2006Q1
1,AA,2.1,2006-03-31,2006-04-10,EPS,6,1.38,1.41,15.0,1,...,869.811,877.006,7797.0,35771.0,12,C,INDL,STD,D,2006Q2
2,AA,2.1,2006-03-31,2006-04-10,EPS,6,1.38,1.41,15.0,1,...,867.590,873.494,7631.0,36304.0,12,C,INDL,STD,D,2006Q3
3,AA,2.1,2006-03-31,2006-04-10,EPS,6,1.38,1.41,15.0,1,...,867.331,873.059,7840.0,37183.0,12,C,INDL,STD,D,2006Q4
4,AA,2.1,2006-03-31,2006-04-10,EPS,6,1.38,1.41,15.0,1,...,868.825,875.753,7908.0,38021.0,12,C,INDL,STD,D,2007Q1


# Step 1: Prepare the Dataset
First, make sure that the final_dataset we merged contains the columns necessary for calculating SUE:

- For SUE1: We need epspxq (earnings per share) from Compustat, fyearq, and fqtr.
- For SUE2: We use the adjusted earnings (epsfxq), excluding special items.
- For SUE3: We need the median analyst forecast (medest) and the actual earnings (actual) from IBES.
- 
# Step 2: Calculate SUE1
SUE1 is calculated using a seasonal random walk model. Essentially, it measures how earnings this quarter compare to the earnings of the same quarter in the previous year, standardized by their variance.

In [16]:
# Sort data to make it easier to reference previous year's earnings
final_dataset = final_dataset.sort_values(by=['gvkey', 'fyearq', 'fqtr'])

# Calculate SUE1: using Seasonal Random Walk Model
# Calculate lagged earnings (previous year's same quarter earnings)
final_dataset['epspxq_lagged'] = final_dataset.groupby(['gvkey', 'fqtr'])['epspxq'].shift(1)

# Calculate SUE1 as (current EPS - previous year's EPS) / standard deviation
final_dataset['epspxq_std'] = final_dataset.groupby(['gvkey', 'fqtr'])['epspxq'].transform('std')
final_dataset['sue1'] = (final_dataset['epspxq'] - final_dataset['epspxq_lagged']) / final_dataset['epspxq_std']

# Display the first few rows to verify
final_dataset[['gvkey', 'fyearq', 'fqtr', 'epspxq', 'epspxq_lagged', 'epspxq_std', 'sue1']].head()


,gvkey,fyearq,fqtr,epspxq,epspxq_lagged,epspxq_std,sue1
898632,001004,2005,3.0,0.27,NaN,0.204921,NaN
898688,001004,2005,3.0,0.27,0.27,0.204921,0.0
898744,001004,2005,3.0,0.27,0.27,0.204921,0.0
898800,001004,2005,3.0,0.27,0.27,0.204921,0.0
898856,001004,2005,3.0,0.27,0.27,0.204921,0.0


# Step 3: Calculate SUE2 (Excluding Special Items)

SUE2 is calculated similarly to SUE1, but uses earnings excluding special items (epsfxq instead of epspxq). The idea is to get a more stable measure by excluding volatile, non-recurring items.

In [17]:
# Calculate lagged earnings excluding special items
final_dataset['epsfxq_lagged'] = final_dataset.groupby(['gvkey', 'fqtr'])['epsfxq'].shift(1)

# Calculate SUE2 as (current EPS excluding special items - previous year's EPS excluding special items) / standard deviation
final_dataset['epsfxq_std'] = final_dataset.groupby(['gvkey', 'fqtr'])['epsfxq'].transform('std')
final_dataset['sue2'] = (final_dataset['epsfxq'] - final_dataset['epsfxq_lagged']) / final_dataset['epsfxq_std']

# Display the first few rows to verify
final_dataset[['gvkey', 'fyearq', 'fqtr', 'epsfxq', 'epsfxq_lagged', 'epsfxq_std', 'sue2']].head()


,gvkey,fyearq,fqtr,epsfxq,epsfxq_lagged,epsfxq_std,sue2
898632,001004,2005,3.0,0.24,NaN,0.20305,NaN
898688,001004,2005,3.0,0.24,0.24,0.20305,0.0
898744,001004,2005,3.0,0.24,0.24,0.20305,0.0
898800,001004,2005,3.0,0.24,0.24,0.20305,0.0
898856,001004,2005,3.0,0.24,0.24,0.20305,0.0


# Step 4: Calculate SUE3 (Based on Analyst Forecasts)
SUE3 is calculated by comparing the median analyst forecast (medest) to the actual earnings (actual). The earnings surprise is standardized by the standard deviation of analyst forecasts or earnings.

In [18]:
# Calculate SUE3 based on analyst forecast
# Calculate standard deviation of analyst estimates if available, otherwise use actual earnings standard deviation
final_dataset['actual_std'] = final_dataset.groupby(['gvkey', 'fqtr'])['actual'].transform('std')

# Calculate SUE3 as (actual - median estimate) / standard deviation of actual earnings
final_dataset['sue3'] = (final_dataset['actual'] - final_dataset['medest']) / final_dataset['actual_std']

# Display the first few rows to verify
final_dataset[['gvkey', 'fyearq', 'fqtr', 'actual', 'medest', 'actual_std', 'sue3']].head()


,gvkey,fyearq,fqtr,actual,medest,actual_std,sue3
898632,001004,2005,3.0,0.2698,0.27,0.193329,-0.001035
898688,001004,2005,3.0,0.2698,0.23,0.193329,0.205867
898744,001004,2005,3.0,0.2698,0.22,0.193329,0.257592
898800,001004,2005,3.0,0.2698,0.21,0.193329,0.309317
898856,001004,2005,3.0,0.2698,0.21,0.193329,0.309317


In [25]:
from Constants import Constants as const
import os
final_dataset.to_pickle(os.path.join(const.TEMP_PATH, '20241006_sue123_data.pkl'))

In [36]:
# List of columns to drop from the final dataset
columns_to_drop = [
    'epspxq_lagged', 'epsfxq_lagged', 'epspxq_std', 'epsfxq_std', 
    'actual_std', 'event_window', 'rdq_plus_2', 'next_rdq_plus_1'
]

final_dataset.drop_duplicates(subset=['gvkey', 'fyearq', 'fqtr'], inplace=True, keep='last')
final_dataset.replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop the columns
final_dataset_reduced = final_dataset.drop(columns=columns_to_drop, errors='ignore')
final_dataset_reduced.dropna(subset=['sue1', 'sue2', 'sue3'], inplace=True, how='all')

# Verify the first few rows of the reduced dataset
print("Columns retained in the reduced dataset:")
print(final_dataset_reduced.columns)
final_dataset_reduced.head()


Columns retained in the reduced dataset:
Index(['ticker', 'actual', 'fpedats', 'anndats_act', 'measure', 'fpi',
       'meanest', 'medest', 'numest', 'usfirm', 'permno', 'namedt',
       'nameenddt', 'gvkey', 'linkdt', 'linkenddt', 'fyearq', 'fqtr', 'conm',
       'datadate', 'rdq', 'epsfxq', 'epspxq', 'prccq', 'ajexq', 'spiq',
       'cshoq', 'cshprq', 'cshfdq', 'saleq', 'atq', 'fyr', 'consol', 'indfmt',
       'datafmt', 'popsrc', 'datafqtr', 'sue1', 'sue2', 'sue3'],
      dtype='object')


,ticker,actual,fpedats,anndats_act,measure,fpi,meanest,medest,numest,usfirm,...,atq,fyr,consol,indfmt,datafmt,popsrc,datafqtr,sue1,sue2,sue3
917336,AIR,0.64,2019-11-30,2019-12-19,EPS,7,0.63,0.63,5.0,1,...,938.151,5,C,INDL,STD,D,2005Q3,0.0,0.0,0.051725
917337,AIR,0.64,2019-11-30,2019-12-19,EPS,7,0.63,0.63,5.0,1,...,978.819,5,C,INDL,STD,D,2005Q4,0.0,0.0,0.051725
917338,AIR,0.64,2019-11-30,2019-12-19,EPS,7,0.63,0.63,5.0,1,...,978.803,5,C,INDL,STD,D,2006Q1,0.0,0.0,0.051725
917339,AIR,0.64,2019-11-30,2019-12-19,EPS,7,0.63,0.63,5.0,1,...,1001.031,5,C,INDL,STD,D,2006Q2,0.0,0.0,0.051725
917340,AIR,0.64,2019-11-30,2019-12-19,EPS,7,0.63,0.63,5.0,1,...,1010.849,5,C,INDL,STD,D,2006Q3,0.0,0.0,0.051725


In [38]:
# Convert data types to reduce memory usage
final_dataset_reduced['fyearq'] = final_dataset_reduced['fyearq'].astype('int32')
final_dataset_reduced['gvkey'] = final_dataset_reduced['gvkey'].astype('category')
final_dataset_reduced['fqtr'] = final_dataset_reduced['fqtr'].astype('int8')
final_dataset_reduced['sue1'] = final_dataset_reduced['sue1'].astype('float32')
final_dataset_reduced['sue2'] = final_dataset_reduced['sue2'].astype('float32')
final_dataset_reduced['sue3'] = final_dataset_reduced['sue3'].astype('float32')

# Check the reduced dataset memory usage
print("Memory usage after type conversion:")
print(final_dataset_reduced.info(memory_usage='deep'))


Memory usage after type conversion:
<class 'pandas.core.frame.DataFrame'>
Index: 168250 entries, 917336 to 21296084
Data columns (total 40 columns):
 #   Column       Non-Null Count   Dtype   
---  ------       --------------   -----   
 0   ticker       168250 non-null  object  
 1   actual       148461 non-null  float64 
 2   fpedats      168250 non-null  object  
 3   anndats_act  168250 non-null  object  
 4   measure      168250 non-null  object  
 5   fpi          168250 non-null  object  
 6   meanest      168142 non-null  float64 
 7   medest       168142 non-null  float64 
 8   numest       168250 non-null  float64 
 9   usfirm       168250 non-null  int64   
 10  permno       168250 non-null  int64   
 11  namedt       168250 non-null  object  
 12  nameenddt    168250 non-null  object  
 13  gvkey        168250 non-null  category
 14  linkdt       168250 non-null  object  
 15  linkenddt    76166 non-null   object  
 16  fyearq       168250 non-null  int32   
 17  fqtr      

In [39]:
final_dataset_reduced.to_pickle(os.path.join(const.TEMP_PATH, '20241006_sue123_data.pkl'))


# Convert quarterly SUE to annual SUE
Group the quarterly data to create annual SUE values for each firm and each fiscal year.

In [40]:
# Import necessary libraries
import pandas as pd

# Filter dataset to ensure each firm-year has four quarters
complete_firm_years = final_dataset_reduced.groupby(['gvkey', 'fyearq']).filter(lambda x: len(x) == 4)

# Group by gvkey and fiscal year to calculate annual SUE values
# Use mean as an aggregation for annual SUE, but you could also use sum depending on the context
annual_sue = complete_firm_years.groupby(['gvkey', 'fyearq']).agg(
    annual_sue1=('sue1', 'mean'),
    annual_sue2=('sue2', 'mean'),
    annual_sue3=('sue3', 'mean')
).reset_index()

# Show the first few rows of the annual SUE dataset
annual_sue.head()


C:\Users\wangy\AppData\Local\Temp\ipykernel_18516\1492738108.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  complete_firm_years = final_dataset_reduced.groupby(['gvkey', 'fyearq']).filter(lambda x: len(x) == 4)
C:\Users\wangy\AppData\Local\Temp\ipykernel_18516\1492738108.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  annual_sue = complete_firm_years.groupby(['gvkey', 'fyearq']).agg(


,gvkey,fyearq,annual_sue1,annual_sue2,annual_sue3
0,001004,2006,0.0,0.0,0.051725
1,001004,2007,0.0,0.0,0.051725
2,001004,2008,0.0,0.0,0.051725
3,001004,2009,0.0,0.0,0.051725
4,001004,2010,0.0,0.0,0.051725


In [42]:
annual_sue.to_pickle(os.path.join(const.TEMP_PATH, '20241006_sue123_data_annual.pkl'))
